# 1. Install and Import Dependencies

In [31]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()
import pickle
import joblib

# std = StandardScaler()

# 2. Load the data

In [15]:
df = pd.read_csv("datasets/Crop_recommendation.csv") 

# 3. Pre-processing the data

In [16]:
x = df.drop(columns="label")
y = pd.DataFrame(df['label'],columns=['label'])
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, train_size = .75)
print(y_train)
y_train["label"] = le.fit_transform(y_train["label"])
# 
y_test["label"] = le.transform(y_test["label"])
# y_test

          label
1975     cotton
1287     grapes
760   blackgram
267    chickpea
2007       jute
...         ...
1033     banana
1731     papaya
763   blackgram
835      lentil
1653     orange

[1650 rows x 1 columns]


In [17]:
x_train

,N,P,K,temperature,humidity,ph,rainfall
1975,134,56,18,23.808346,83.919026,6.691268,70.973583
1287,29,122,196,41.948657,81.155952,5.638328,73.068630
760,25,68,19,29.399827,64.255107,7.108450,67.476773
267,41,69,82,20.023815,16.632945,6.715587,68.978065
2007,70,43,40,24.355641,88.803910,6.176860,169.116803
...,...,...,...,...,...,...,...
1033,102,71,48,28.654563,79.286937,5.695268,102.463378
1731,34,68,51,27.347349,94.177567,6.687088,40.351531
763,35,64,15,28.474423,63.536045,6.500145,69.527441
835,39,65,23,25.434598,69.126134,7.685959,41.026829


In [18]:
y_train

,label
1975,6
1287,7
760,2
267,3
2007,8
...,...
1033,1
1731,17
763,2
835,10


In [19]:
y_train = to_categorical(y_train, 22)
y_test = to_categorical(y_test, 22)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 4. Model

In [20]:
def crop_predection__model():

    # Define the model
    model = tf.keras.models.Sequential([ 
      tf.keras.layers.Dense(32 , activation='relu'),
      tf.keras.layers.Dense(64 , activation='relu'),
      tf.keras.layers.Dense(22 , activation='softmax'),
  
    ]) 


    # Compile the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [21]:
model =crop_predection__model()

In [22]:
history = model.fit(x_train, y_train, epochs=5)

Epoch 1/5
52/52 [==============================] - 1s 2ms/step - loss: 13.2610 - accuracy: 0.1485
Epoch 2/5
52/52 [==============================] - 0s 2ms/step - loss: 1.6416 - accuracy: 0.5200
Epoch 3/5
52/52 [==============================] - 0s 2ms/step - loss: 0.6487 - accuracy: 0.7842
Epoch 4/5
52/52 [==============================] - 0s 2ms/step - loss: 0.4437 - accuracy: 0.8618
Epoch 5/5
52/52 [==============================] - 0s 2ms/step - loss: 0.3636 - accuracy: 0.8879


# 5. Testing

In [23]:
model.evaluate(x_test, y_test, batch_size=50)  

11/11 [==============================] - 0s 2ms/step - loss: 0.3174 - accuracy: 0.9055


[0.3174194395542145, 0.9054545164108276]

In [24]:
pred = model.predict(x_train.iloc[[0]])
pred

array([[5.5004503e-09, 1.6931834e-03, 6.7399524e-06, 1.1312662e-11,
        6.4946573e-15, 2.4107056e-04, 8.9060384e-01, 3.2606606e-10,
        2.0033778e-05, 2.6843587e-09, 3.0474973e-07, 1.0706480e-01,
        4.1629220e-14, 1.5300918e-04, 7.9405827e-06, 1.8593507e-04,
        8.3366120e-07, 1.8605505e-07, 1.4185349e-05, 1.4711353e-06,
        1.2566281e-08, 6.4257101e-06]], dtype=float32)

In [27]:
max3_values = np.argsort(pred[0])[::-1]
max3_values

array([ 6, 11,  1,  5, 15, 13,  8, 18, 14,  2, 21, 19, 16, 10, 17, 20,  0,
        9,  7,  3, 12,  4], dtype=int64)

In [28]:
le.inverse_transform(max3_values)

array(['cotton', 'maize', 'banana', 'coffee', 'muskmelon', 'mothbeans',
       'jute', 'pigeonpeas', 'mungbean', 'blackgram', 'watermelon',
       'pomegranate', 'orange', 'lentil', 'papaya', 'rice', 'apple',
       'kidneybeans', 'grapes', 'chickpea', 'mango', 'coconut'],
      dtype=object)

# Pickle file

In [144]:
# pickle.dump(model, open("model.pkl", "wb"))

INFO:tensorflow:Assets written to: ram://d242385e-2dcd-4293-960e-9bdee99c523c/assets


In [139]:
# model.save("saved_model/crop_predection__model")

INFO:tensorflow:Assets written to: saved_model/crop_predection__model\assets


In [32]:
# pickle.dump(model , open("model.pkl" , "wb"))
joblib.dump(model, 'model2.pkl')

INFO:tensorflow:Assets written to: ram://96a57744-88b0-4558-8534-922f760f261e/assets


['model2.pkl']